DEPENDENCIES

In [17]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine , inspect
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

EXTRACT

In [2]:
indeed  = 'https://www.indeed.com/jobs?q=data+analyst&l=#'
response = requests.get(indeed).text

In [3]:
soup = bs(response, 'html.parser')

In [6]:
jobCards = soup.find_all('div', class_= 'jobsearch-SerpJobCard')

TRANSFORM

In [7]:
jobList =[]
# Loop through returned results
for job in jobCards:
    # Error handling
    try:
        # Identify and return title of listing
        title = job.find('a', class_='jobtitle').text.strip()
        company = job.find('span', class_='company').text.strip()
        location = job.find('div', class_ = 'location accessible-contrast-color-location').text.strip()
      

        # Print results only if title, price, and link are available
        if (title and company and location):

           # Dictionary to be inserted into list
            post = {
                'job_title': title,
                'company': company,
                'location': location
            }
            jobList.append(post)
            
    except AttributeError as e:
        print(e)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [8]:
df1 = pd.DataFrame(jobList)  

In [9]:
df1.head(5)

,job_title,company,location
0,Data Analyst,Elite Manufacturing & Packaging,"Vineyard, UT 84042"
1,Jr. Data Analyst,Pactera Technologies,"Texas, VT"
2,Data Analyst - 9867780,Ursus,"McLean, VA 22102"
3,Data Analyst,Merric Millwork and Seating,"O'Fallon, MO 63366"
4,"Analyst, Data I - 2nd shift",FedEx Supply Chain,"York, PA 17408"


LOAD

In [15]:
# Create the posting class (the posting table that will be in the db)
class Posting(Base):
    __tablename__ = 'posting'
    id = Column(Integer, primary_key=True)
    job_title = Column(String(255))
    company = Column(String(255))
    location = Column(String(255))


C:\Users\mharg\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Posting, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


InvalidRequestError: Table 'posting' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [16]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///job_postings.db')
conn = engine.connect()
Base.metadata.create_all(engine)  #this should create the table based on the classes defined
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [12]:
# Iterate over df and add rows as records to table

for i in df1.itertuples():

    try:
        record = Posting(job_title = i.job_title, company = i.company, location = i.location )
        session.add(record)
        session.commit()
 
    except:
        pass
    


TEST


In [18]:
#Insepect the database
inspector = inspect(engine)
inspector.get_table_names()

['posting']

In [19]:
columns = inspector.get_columns('posting')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
job_title VARCHAR(255)
company VARCHAR(255)
location VARCHAR(255)


In [20]:
postings = session.query(Posting)
for job in postings:
  print(job.job_title)

Data Analyst
Jr. Data Analyst
Data Analyst - 9867780
Data Analyst
Analyst, Data I - 2nd shift
Data Analyst
Data Analyst
Customer Support Specialist - Data Analyst
CCoE CM Cyber Sit. Understanding Data Analyst


RESET DataBase

In [ ]:
# Use this to clear out the db when necessary

#Base.metadata.drop_all(engine)

Troubleshooting Resources:

In [ ]:
# If needed to explore the soup object

#jobCard = soup.find('div', class_= 'jobsearch-SerpJobCard')
#print(jobCard.prettify())